# BASIC INTRODUCTION TO SIGMA JS IN JUPYTER NOTEBOOK AND NETWORK SCIENCE

## Introduction 

Please refer to my blog in the read me file which introduces networks and sigma js before reading the code below , as it will give you a clear understanding of the methods involved in the code.


In [10]:
# load Preliminaries
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [11]:
# loading the engines locally along with plug-in.
HTML('''
<script src="./sigma.min.js"></script>
<script src="./sigma-add-method-neighbors.js"></script>

''')

## NETWORK SCIENCE .

Let's create a random network connection with nodes.Remember to keep them in dictionary format.

In [12]:
random.seed(42)

n_nodes = 40
n_edges = 200

graph_data = { 'nodes': [], 'edges': [] }

for i in range(n_nodes):
    graph_data['nodes'].append({
            "id": "n" + str(i),
            "label": "n" + str(i),
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": random.uniform(0.2,1)
        })

for j in range(n_edges):
    x_center = random.uniform(0,1)
    y_center = random.uniform(0,1)
    x_dist = random.uniform(0.1,0.5)
    y_dist = random.uniform(0.2,0.5)
    neighborhood = []
    for node in graph_data['nodes']:
        if abs(node['x'] - x_center) < x_dist:
            if abs(node['y'] - y_center) < y_dist:
                neighborhood.append(int(node['id'].replace('n','')))
    if len(neighborhood) >= 2:
        ends = random.sample(neighborhood,2)
        graph_data['edges'].append({
                "id": "e" + str(j),
                "source": "n" + str(ends[0]),
                "target": "n" + str(ends[1])
            })

In [13]:
pd.DataFrame(graph_data['nodes']).head()

id label      size         x         y
0  n0    n0  0.420023  0.639427  0.025011
1  n1    n1  0.741360  0.223211  0.736471
2  n2    n2  0.537537  0.892180  0.086939
3  n3    n3  0.604284  0.029797  0.218638
4  n4    n4  0.719908  0.026536  0.198838

In [14]:
pd.DataFrame(graph_data['edges']).head()

id source target
0  e0     n5    n12
1  e1    n25     n1
2  e2    n10    n20
3  e3     n6    n29
4  e4    n15    n17

Now Let's get the java script.

In [15]:
js_text_template = Template('''
    
    var g = $graph_data ;

s = new sigma({graph: g, container: '$container', settings: { defaultNodeColor: '#ec5148'} });

s.graph.nodes().forEach(function(n) {
  n.originalColor = n.color;
});
s.graph.edges().forEach(function(e) {
  e.originalColor = e.color;
});

s.bind('clickNode', function(e) {
  var nodeId = e.data.node.id,
      toKeep = s.graph.neighbors(nodeId);
  toKeep[nodeId] = e.data.node;

  s.graph.nodes().forEach(function(n) {
    if (toKeep[n.id])
      n.color = n.originalColor;
    else
      n.color = '#eee';
  });

  s.graph.edges().forEach(function(e) {
    if (toKeep[e.source] && toKeep[e.target])
      e.color = e.originalColor;
    else
      e.color = '#eee';
  });

  s.refresh();
});

s.bind('clickStage', function(e) {
  s.graph.nodes().forEach(function(n) {
    n.color = n.originalColor;
  });

  s.graph.edges().forEach(function(e) {
    e.color = e.originalColor;
  });

  s.refresh();
});


''')

In [16]:
js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div'})


In [17]:
html_template = Template('''
<div id="graph-div" style="height:800px"></div>
<script> $js_text </script>
''')


In [18]:
HTML(html_template.substitute({'js_text': js_text}))